In [28]:
from ipywidgets import Layout, Label, VBox, interact, fixed, FloatSlider

from ipyleaflet import Map, Choropleth, LayerGroup, LayersControl
import geopandas
import numpy as np
import json
from branca.colormap import linear

import map_methods
import math


In [29]:
def get_macedonia_map():
    center=(41.6086, 21.7453)

    m = Map(
        layers=(basemap_to_tiles(basemaps.OpenStreetMap.Mapnik), ),
        center=center,
        zoom=8,
        scroll_wheel_zoom=True,
        layout=Layout(width='100%', height='600px')
    )
    
    return m

In [30]:
covid_mk_cities = geopandas.read_file("covid.geojson")

In [31]:
scale = np.log2((covid_mk_cities['count'] / (covid_mk_cities['population'] / 2000) + 1))
idd = map(str, covid_mk_cities.index)
covid_active = dict(zip(idd, scale))
covid_mk_cities_geo_json = json.loads(covid_mk_cities.to_json())

In [32]:
colors = Choropleth(
    geo_data=covid_mk_cities_geo_json,
    choro_data=covid_active,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.5, 'weight': 0.05},
    hover_style={'fillColor': 'grey' , 'fillOpacity': 0.3}
)

label = Label(layout=Layout(width='100%'))

def hover_handler(event=None, feature=None, id=None, properties=None):
#     label.value = properties['geounit']
    label.value = properties['id'] + " " + str(properties['count'])

In [33]:
def draw_map(color_min):
    m = get_macedonia_map()
    # hospitals = map_methods.marker_layer(d=covid_hospitals, 
    #                              cols=["count_x", "count_y", "count"], 
    #                              names=["Заразени", "Излечени" ,"Мртви"],
    #                             get_marker=get_marker,
    #                             hide_marker=hide_marker,
    #                             lng='Longitude',
    #                             lat='Latitude')
    # layer_group = LayerGroup(layers=(colors))

    colors.on_hover(hover_handler)
    colors.value_min = color_min
    m.add_layer(colors)
    # m.add_layer(geo_data)
    m.add_control(LayersControl())


    return VBox([m, label])
    

color_slicer = FloatSlider(min=colors.value_min, max=colors.value_max, continuous_update=False)
    
interact(draw_map, color_min=color_slicer)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='color_min', max=11.43268756…

<function __main__.draw_map(color_min)>